In [142]:
# __author__: CH
# __date__: Apr 19

In [143]:
import pandas as pd
import numpy as np
import os.path
import geojson 
import json
import urllib2
import geopandas as gpd
import pysal
import pickle
from shapely import geometry

cwd = os.getcwd()

In [2]:
# get pluto data
pluto_bk = gpd.read_file(os.path.join(os.path.dirname(cwd), 'data','bk_mappluto_16v2', 'BKMapPLUTO.shp'))
pluto_mn = gpd.read_file(os.path.join(os.path.dirname(cwd), 'data','mn_mappluto_16v2', 'MNMapPLUTO.shp'))
pluto_qn = gpd.read_file(os.path.join(os.path.dirname(cwd), 'data','qn_mappluto_16v2', 'QNMapPLUTO.shp'))
pluto_si = gpd.read_file(os.path.join(os.path.dirname(cwd), 'data','si_mappluto_16v2', 'SIMapPLUTO.shp'))

pmap = pd.concat([pluto_bk, pluto_mn, pluto_qn, pluto_si], ignore_index=True)
pmap.geometry = pmap.geometry.to_crs(epsg = 4326)

print '# BBL:', len(pmap.BBL)

# BBL: 767552


In [3]:
# read in FQ biz records
data = pd.read_csv(os.path.join(os.path.dirname(cwd), 'data', 'data', 'output', 'chunk_0.csv'))

# convert string to geometry point
def convert_str_point(string):
    x = float(string.split('POINT')[1].split(' ')[1][1:])
    y = float(string.split('POINT')[1].split(' ')[2][:-1])
    return geometry.Point((x,y))

# add in a new column in data
data['true_geo'] = map(convert_str_point, data.geometry)
# turn into a geodataframe
data = gpd.GeoDataFrame(data, geometry = 'true_geo')

In [97]:
def findbbl(biz_list, BBLdata):
    '''
    for faster scan
    define each geometry point's BBL
    get the first BBL record
    '''
    # using R-Tree algorithm
    precise_matches = []
    spatial_index = BBLdata.sindex
    for i, point in enumerate(biz_list.true_geo):
        pbuff = point.buffer(0.00000001)
        possible_matches_index = list(spatial_index.intersection(pbuff.bounds))
        possible_matches = BBLdata.iloc[possible_matches_index, :]
        try:
            #precise_matches.append(possible_matches[possible_matches.geometry.contains(biz_list['true_geo'][i])].iloc[0,6])
            pre_match_set = possible_matches[possible_matches.geometry.contains(biz_list['true_geo'][i])]
            if len(pre_match_set) > 0:
                precise_matches.append(pre_match_set.iloc[0,6])
            else:
                precise_matches.append(np.nan)
        except ValueError:
            precise_matches.append(np.nan)

    return precise_matches

In [87]:
def easyfind(x):
    '''
    without geo threshold
    slower
    '''
    find = BBLdata[BBLdata.geometry.contains(x)]
    if len(find) > 0:
        return find.iloc[0,6]
    else:
        return np.nan

In [98]:
# Compare above two functions
bbllist = findbbl(data, pmap)
bbllist2 = map(easyfind, data['true_geo'])

In [108]:
bbllist_all = pd.DataFrame([bbllist, bbllist2]).T
bbllist_all

,0,1
0,1.002360e+09,1.002360e+09
1,1.002360e+09,1.002360e+09
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,1.005400e+09,1.005400e+09


### Seems they return the same result but why protions of NaN got so high?
### Take a look at the records:

In [135]:
nan_index = bbllist_all.loc[set(bbllist_all.index) - set(bbllist_all.dropna().index), :].index

In [140]:
# Ha no idea what's going on
data.loc[nan_index, :]

,Unnamed: 0,rating,stats.checkinsCount,stats.tipCount,stats.usersCount,stats.visitsCount,location.lat,location.lng,location.postalCode,id,geometry,fs_cat,true_geo
2,2,5.8,6899,19,2449,8496,40.729352,-74.006925,10014.0,4b2b9539f964a520e0b724e3,POINT (-74.00692507282733 40.72935205409404),NaN,POINT (-74.00692507282733 40.72935205409404)
3,3,7.7,113,5,283,510,40.838350,-73.939963,10032.0,53f045ba498ec9e735cd769d,POINT (-73.93996314107633 40.83835015785895),NaN,POINT (-73.93996314107633 40.83835015785895)
4,4,NaN,285,1,161,319,40.628321,-74.029090,11209.0,4d211b82d7b0b1f7ef8b1a9f,POINT (-74.02909 40.628321),NaN,POINT (-74.02909 40.628321)
5,5,7.4,568,16,761,1479,40.829271,-73.948482,10031.0,545c1ceb498e0ebdeeb0c3c2,POINT (-73.94848174669116 40.82927139337374),NaN,POINT (-73.94848174669116 40.82927139337374)
6,6,8.5,617,21,457,950,40.749381,-73.888562,11372.0,4b76e7f1f964a52025692ee3,POINT (-73.88856230389622 40.74938136280484),NaN,POINT (-73.88856230389622 40.74938136280484)
7,7,6.0,548,6,293,566,40.799525,-73.968308,10025.0,4bcbd62afb84c9b60ba21f3e,POINT (-73.96830797195435 40.79952463781577),NaN,POINT (-73.96830797195435 40.79952463781577)
8,8,NaN,52,2,55,93,40.684703,-73.844810,11416.0,4c539b4c1b46c9b680e936cd,POINT (-73.84480956099328 40.6847034087475),NaN,POINT (-73.84480956099328 40.6847034087475)
10,10,8.6,10938,112,7199,14423,40.716935,-73.999746,10013.0,4b55cd09f964a520acf027e3,POINT (-73.99974616246784 40.7169346902617),NaN,POINT (-73.99974616246784 40.7169346902617)
12,12,7.0,327,1,339,662,40.767161,-73.738468,NaN,51ccbc60498eb28875eaf9ce,POINT (-73.73846830085699 40.76716092119024),NaN,POINT (-73.73846830085699 40.76716092119024)
16,16,6.6,57,1,45,68,40.881249,-73.878944,10467.0,4b4f7901f964a520090827e3,POINT (-73.87894431325324 40.88124851192866),NaN,POINT (-73.87894431325324 40.88124851192866)
